# Предсказание банковских транзакций

**Описание задачи**<br/>
Одним из самых ценных источников информации о клиенте являются данные о банковских транзакциях.</br>
В этом соревновании участникам предлагается предсказать будущие траты клиента, используя информацию о совершенных тратах.</br>

**Полезная информация**</br>
Для понимания представленных данных будет полезна следующая <a href="https://www.banki.ru/wikibank/mcc-kod/">*статья*</a></br>

<a href="https://mcc-codes.ru/code">Описание MCC-кодов.</a></br>

**BaseLine**</br>
Для быстрого старта можно использовать решение, приведенное ниже:</br>
https://www.kaggle.com/prampampam/baseline-popular-transactions

**Формат решения задачи**</br>
Решение ожидаем в виде файла csv. Пример: submission_file.csv из <a href="https://www.kaggle.com/competitions/alfabank-campus/data">раздела Data.</a></br>

**Вопросы**</br>
https://www.kaggle.com/competitions/alfabank-campus/discussion/442418

**Метрика качества**</br>
<a href="https://habr.com/ru/companies/econtenta/articles/303458">**MAP@10**</a></br>

**Описание набора данных**




**File descriptions**</br>
1. df_train.csv — данные для обучения предиктиного алгоритма
2. df_test.csv — тестовый датасет
3. submission_file.csv — пример файла сабмита клиента

**Data fields** </br>
1. df_train.csv
   - data - история трат клиентов (последовательность mcc-кодов в хронологическом порядке);
   - target - последовательность будущих трат клиента.
3. df_test.csv
   - Id - идентификатор клиента;
   - история трат клиентов (последовательность mcc-кодов в хронологическом порядке). 
5. submission_file.scv
   - Id - идентификатор клиента;
   - Predicted - предсказание следующих 10 транзакций клиента.

## Подготовка и анализ

Общая концепция: 
1. Избавиться от выбросов
2. Определить количество значений у каждого id, построить ящик с усами, оставить только ниболее большое количество значений или то количество значений, соответствующее выборке с предсказаниями.
3. Обучить модель на данных
4. Предсказать значения для нескольких случаев(10)

Произведём загрузку данных, библиотек и их компонентов

In [1]:
import pandas as pd
from IPython.display import display 

Определим функцию для загрузки файла

In [2]:
def download_file(name, sep=';'):
    try:
        data = pd.read_csv(name, index_col ='Id', sep=sep)
        return data
    except Exception as _:
        print('Загрузка файла вызвала ошибку\n', _)

Получим наборы данных для изучения

In [3]:
df_train = download_file('df_train.csv')
df_test = download_file('df_test.csv')
submission_baseline_2 = download_file('submission_baseline_2.csv', sep=',')

Приведение значений из строчного формата в чиловой

In [4]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

Изучим получившиеся наборы данных

In [5]:
df_train.head()

,Data,Target
Id,,
0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."


Определить количество значений у каждого id, построить ящик с усами, оставить только ниболее большое количество значений или то количество значений, соответствующее выборке с предсказаниями.

In [6]:
# d_train = {}
# for el in df_train.loc[0, 'Data']:
#     if el not in d_train.keys():
#         d_train[el] = 1
#     else:
#         d_train[el] += 1

# #d_train
# for key, val in d_train.items():
#     df_train.loc[:, key] = val
#     print(key, val)

In [23]:
def make_columns(row):
    d_train = {}
    for el in row['Data']:
        if el not in d_train.keys():
            d_train[el] = 1
        else:
            d_train[el] += 1

    
    # display(pd.DataFrame([d_train.values()], columns=d_train.keys()))
    new_data = pd.DataFrame([d_train.values()], columns=d_train.keys())
    
    # for key, val in d_train.items():
    #     row[key] = val
    
    # print(key, val)
    # print(row)
    # # row[2] = d_train.keys()
    
    return row#d_train.keys()
    

    

In [24]:
df_train.head(1).apply(make_columns, axis=1)

dict_values([81, 1, 75, 28, 29, 2])

dict_keys([4814, 6010, 6011, 5411, 5311, 5921])

,4814,6010,6011,5411,5311,5921
0,81,1,75,28,29,2


,Data,Target
Id,,
0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."


In [ ]:
df_train.head()

In [ ]:
for el in set(df_train.loc[0, 'Data']):
    print(df_train.loc[df_train['Data'] == el, 'Data'])

In [ ]:
[i for i in range(len(df_train.loc[0, 'Data'])) if df_train.loc[0, 'Data'][i] == 4814]

In [ ]:
df_train.loc[0, 'Data'][0]

Определим функцию для отображения первичной информации

In [ ]:
def first_info(name):
    display(name.head())
    display(name.info())

In [ ]:
top10_codes = df_train['Data'].explode().value_counts().head(10)
top10_codes

In [ ]:
df_train['Data'].explode().value_counts().reset_index().hist(bins=20)

In [ ]:
boxplot_train = df_train['Data'].explode().value_counts().reset_index().boxplot('Data', return_type='dict')
boxplot_train;

Получим граничные значения

In [ ]:
[item.get_ydata()[1] for item in boxplot_train['whiskers']] 

*Значения, которые не соответствуют диапазону удалим*

Общая концепция: 
1. Избавиться от выбросов
2. Определить количество значений у каждого id, построить ящик с усами, оставить только ниболее большое количество значений или то количество значений, соответствующее выборке с предсказаниями.
3. Обучить модель на данных
4. Предсказать значения для нескольких случаев(10)

In [ ]:
first_info(submission_baseline_2)

## fgfg

In [ ]:
def str_info(num):
    print(len(df_train.loc[num, 'Data']))
    print(len(df_test.loc[num, 'Data']))
    print(len(submission_baseline_2.loc[num, "Predicted"][1:-1].split(' ')))

str_info(0)

In [ ]:
# for i in range(5):
#     print(i+1)
#     str_info(i)
#     print()

In [ ]:
submission_baseline_2.columns